In [2]:
import tkinter as tk
from tkinter import messagebox

class SudokuSolver:
    def __init__(self, board):
        self.board = board
        self.size = 9
        self.box_size = 3
        self.empty_cell = 0
        self.domains = {}
        self.initialize_domains()

    def initialize_domains(self):
        for row in range(self.size):
            for col in range(self.size):
                if self.board[row][col] != self.empty_cell:
                    self.domains[(row, col)] = {self.board[row][col]}
                else:
                    self.domains[(row, col)] = set(range(1, self.size + 1))

    def get_related_cells(self, row, col):
        related = set()
        for c in range(self.size):
            if c != col:
                related.add((row, c))
        for r in range(self.size):
            if r != row:
                related.add((r, col))
        box_start_row = (row // self.box_size) * self.box_size
        box_start_col = (col // self.box_size) * self.box_size
        for r in range(box_start_row, box_start_row + self.box_size):
            for c in range(box_start_col, box_start_col + self.box_size):
                if r != row or c != col:
                    related.add((r, c))
        return related

    def is_consistent(self):
        for i in range(self.size):
            row_vals = set()
            for j in range(self.size):
                if self.board[i][j] != self.empty_cell:
                    if self.board[i][j] in row_vals:
                        return False
                    row_vals.add(self.board[i][j])
            col_vals = set()
            for j in range(self.size):
                if self.board[j][i] != self.empty_cell:
                    if self.board[j][i] in col_vals:
                        return False
                    col_vals.add(self.board[j][i])
            box_vals = set()
            box_row, box_col = (i // self.box_size) * self.box_size, (i % self.box_size) * self.box_size
            for r in range(box_row, box_row + self.box_size):
                for c in range(box_col, box_col + self.box_size):
                    if self.board[r][c] != self.empty_cell:
                        if self.board[r][c] in box_vals:
                            return False
                        box_vals.add(self.board[r][c])
        return True

    def ac3(self):
        queue = [(r, c) for r in range(self.size) for c in range(self.size) if self.board[r][c] != self.empty_cell]
        while queue:
            row, col = queue.pop(0)
            value = self.board[row][col]
            for r, c in self.get_related_cells(row, col):
                if value in self.domains.get((r, c), set()) and len(self.domains.get((r, c), set())) > 1:
                    self.domains[(r, c)].remove(value)
                    if len(self.domains[(r, c)]) == 1:
                        self.board[r][c] = next(iter(self.domains[(r, c)]))
                        queue.append((r, c))
                    elif len(self.domains[(r, c)]) == 0:
                        return False
        return True

    def select_unassigned_cell(self):
        min_domain_size = float('inf')
        selected_cell = None
        for row in range(self.size):
            for col in range(self.size):
                if self.board[row][col] == self.empty_cell:
                    domain_size = len(self.domains.get((row, col), set()))
                    if domain_size < min_domain_size:
                        min_domain_size = domain_size
                        selected_cell = (row, col)
        return selected_cell

    def backtrack(self):
        if all(self.board[r][c] != self.empty_cell for r in range(self.size) for c in range(self.size)):
            return True
        if not self.ac3():
            return False
            
        cell = self.select_unassigned_cell()
        if cell is None:
            return True
            
        row, col = cell
        for value in sorted(self.domains.get((row, col), set())):
            self.board[row][col] = value
            old_domains = {k: v.copy() for k, v in self.domains.items()}
            self.domains[(row, col)] = {value}
            if self.backtrack():
                return True
            self.board[row][col] = self.empty_cell
            self.domains = old_domains
            
        return False

    def solve(self):
        if not self.is_consistent():
            return False
        return self.backtrack()

class SudokuGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Sudoku Solver")
        self.board = [[0] * 9 for _ in range(9)]
        self.entries = [[None] * 9 for _ in range(9)]
        self.create_widgets()

    def create_widgets(self):
        for r in range(9):
            for c in range(9):
                entry = tk.Entry(self.root, width=3, font=('Arial', 24), justify='center')
                entry.grid(row=r, column=c, padx=5, pady=5)
                self.entries[r][c] = entry
        
        solve_button = tk.Button(self.root, text="Solve", command=self.solve_sudoku)
        solve_button.grid(row=10, columnspan=9, pady=10)

    def solve_sudoku(self):
        for r in range(9):
            for c in range(9):
                value = self.entries[r][c].get()
                if value.isdigit() and value != "":
                    self.board[r][c] = int(value)
                else:
                    self.board[r][c] = 0

        solver = SudokuSolver(self.board)
        if solver.solve():
            for r in range(9):
                for c in range(9):
                    self.entries[r][c].delete(0, tk.END)
                    self.entries[r][c].insert(0, solver.board[r][c])
        else:
            messagebox.showinfo("Sudoku Solver", "No solution exists for this Sudoku puzzle.")

if __name__ == "__main__":
    root = tk.Tk()
    gui = SudokuGUI(root)
    root.mainloop()